# 如何追踪大语言模型的令牌使用情况
跟踪 [token](/docs/concepts/tokens/) 使用情况以计算成本是将应用程序投入生产的重要环节。本指南将介绍如何从 LangChain 模型调用中获取这些信息。
:::info 前提条件
本指南假定您熟悉以下概念：
- [大型语言模型（LLMs）](/docs/concepts/text_llms):::
## 使用LangSmith
您可以使用 [LangSmith](https://www.langchain.com/langsmith) 来帮助追踪 LLM 应用中的令牌使用情况。详情请参阅 [LangSmith 快速入门指南](https://docs.smith.langchain.com/)。
## 使用回调函数
有一些特定于API的回调上下文管理器，允许您跨多个调用跟踪令牌使用情况。您需要检查是否针对您使用的特定模型提供了此类集成。
如果您的模型没有此类集成，您可以通过适配 [OpenAI 回调管理器](https://python.langchain.com/api_reference/community/callbacks/langchain_community.callbacks.openai_info.OpenAICallbackHandler.html) 的实现来创建自定义回调管理器。
### OpenAI
让我们首先看一个极其简单的示例：追踪单个聊天模型调用的令牌使用情况。
:::危险
回调处理器目前不支持传统语言模型（例如 `langchain_openai.OpenAI`）的流式令牌计数功能。如需在流式场景中获取支持，请参考聊天模型的相关指南[此处](/docs/how_to/chat_token_usage_tracking)。
好的,我会按照要求将英文翻译成中文,并保持markdown格式一致。以下是一个示例:

# 标题1
## 标题2
### 标题3

这是一个段落文本。

- 无序列表项1
- 无序列表项2
- 无序列表项3

1. 有序列表项1
2. 有序列表项2
3. 有序列表项3

**加粗文本**  
*斜体文本*  
~~删除线文本~~

[链接文本](https://example.com)

![图片描述](image.jpg)

> 引用文本

```python
# 代码块
def hello():
    print("Hello World!")
```

| 表头1 | 表头2 |
|-------|-------|
| 单元格1 | 单元格2 |
| 单元格3 | 单元格4 |

### 单次调用

In [1]:
from langchain_community.callbacks import get_openai_callback
from langchain_openai import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

with get_openai_callback() as cb:
    result = llm.invoke("Tell me a joke")
    print(result)
    print("---")
print()

print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")



Why don't scientists trust atoms?

Because they make up everything.
---

Total Tokens: 18
Prompt Tokens: 4
Completion Tokens: 14
Total Cost (USD): $3.4e-05


### 多次调用
在上下文管理器内的所有操作都将被追踪。以下是一个示例，展示如何用它来依次追踪对链式调用的多次调用。这种方法同样适用于可能需要多步骤执行的智能体。

In [2]:
from langchain_community.callbacks import get_openai_callback
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

template = PromptTemplate.from_template("Tell me a joke about {topic}")
chain = template | llm

with get_openai_callback() as cb:
    response = chain.invoke({"topic": "birds"})
    print(response)
    response = chain.invoke({"topic": "fish"})
    print("--")
    print(response)


print()
print("---")
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")



Why did the chicken go to the seance?

To talk to the other side of the road!
--


Why did the fish need a lawyer?

Because it got caught in a net!

---
Total Tokens: 50
Prompt Tokens: 12
Completion Tokens: 38
Total Cost (USD): $9.400000000000001e-05


## 流式传输
:::危险
`get_openai_callback` 目前不支持传统语言模型（例如 `langchain_openai.OpenAI`）的流式令牌计数。若需在流式场景中正确统计令牌数，可通过以下方式实现：
- 按照[本指南](/docs/how_to/chat_token_usage_tracking)所述使用聊天模型；- 实现一个[自定义回调处理程序](/docs/how_to/custom_callbacks/)，该处理程序使用适当的标记器来统计标记数量；- 使用监控平台，例如 [LangSmith](https://www.langchain.com/langsmith)。:::
请注意，在流式传输环境中使用传统语言模型时，令牌计数不会更新：

In [3]:
from langchain_community.callbacks import get_openai_callback
from langchain_openai import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

with get_openai_callback() as cb:
    for chunk in llm.stream("Tell me a joke"):
        print(chunk, end="", flush=True)
    print(result)
    print("---")
print()

print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")



Why don't scientists trust atoms?

Because they make up everything!

Why don't scientists trust atoms?

Because they make up everything.
---

Total Tokens: 0
Prompt Tokens: 0
Completion Tokens: 0
Total Cost (USD): $0.0
